Complete the exercises below For **Assignment #4**.

Use **Markdown** cells for the non-code answers.

In this assignment we will work with the data underlying the *FiveThirtyEight* article ["Higher Rates Of Hate Crimes Are Tied To Income Inequality"](https://fivethirtyeight.com/features/higher-rates-of-hate-crimes-are-tied-to-income-inequality/).

Load the `tidymodels`, `readr`, and `moderndive` packages in the cell below.

In [1]:
install.packages(c("tidymodels", "readr", "moderndive"))



Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



We can read the data from a **CSV file** at the following URL: [http://bit.ly/2ItxYg3](http://bit.ly/2ItxYg3).

Use the `read_csv` function to read the data into our R session. Call the new table `hate_crimes`.

In [10]:
hate_crimes <- read_csv("hate_crimes.csv")

ERROR: Error in read_csv("hate_crimes.csv"): could not find function "read_csv"


Next, let’s add the high-school degree variable into the mix by creating a scatterplot showing:

- Income on the y-axis (this is the `income` variable)
- Percent of adults 25 or older with a high school degree on the x-axis (this is the `hs` variable)
- The points colored by level of urbanization in a region (this is the variable `urbanization`)


**In addition, add a line of best fit (regression line) for each level of the variable urbanization (one for “low”, one for “high”).**

*Add the regression lines to the plot using the `geom_parallel_slopes` function from the `moderndive` package. This function will draw the regression lines based on fitting a regression model with parallel slopes (i.e., with no interaction between `hs` and `urbanization`).*

In [8]:
library(ggplot2)
library(moderndive)

ggplot(hate_crimes, aes(x = hs, y = income, color = urbanization)) +
  geom_point() +
  geom_parallel_slopes(aes(group = urbanization)) +
  labs(
    title = "Income vs. High School Degree Percentage",
    x = "Percent of Adults 25+ with a High School Degree",
    y = "Income",
    color = "Urbanization Level"
  ) +
  theme_minimal()




Warning message:
“Your system is mis-configured: ‘/var/db/timezone/localtime’ is not a symlink”
Warning message:
“‘/var/db/timezone/localtime’ is not identical to any known timezone file”


ERROR: Error: object 'hate_crimes' not found


❓Which regression line (high urbanization or low urbanization) appears to have the larger intercept?

**Answer:**

In [ ]:
#Low urbanization has the larger intercept.


Now let’s create a second scatterplot using the same variables, but this time draw the regression lines using `geom_smooth(method = "lm")`, which will allow for separate, non-parallel slopes for each urbanization group. 

**Code your scatter plot in the cell below.**

In [11]:
library(ggplot2)

ggplot(hate_crimes, aes(x = hs, y = income, color = urbanization)) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE) +
  labs(
    title = "Income vs. High School Degree Percentage",
    x = "Percent of Adults 25+ with a High School Degree",
    y = "Income",
    color = "Urbanization Level"
  ) +
  theme_minimal()

ERROR: Error: object 'hate_crimes' not found


❓Based on visually comparing the two models shown above, do you think it would be best to run a “parallel slopes” model (i.e. a model that estimates one shared slope for the two levels of urbanization), or a more complex “interaction model” (i.e. a model that estimates a separate slope for the two levels of urbanization)?

**Answer:**

In [18]:
#It would be best to run a more complex interaction model.


Fit the following two regression models that examine the relationship between household `income` (as response variable), and high-school education (`hs`) and `urbanization` as explanatory variables:

1. A parallel slopes model (i.e., no interaction between `hs` and `urbanization`). ❗️Save the data recipe and model under the variables `ps_rec` and `ps_mod`, respectively. 
1. A non-parallel slopes model (i.e., allow `hs` and `urbanization` to interact in your model). ❗️Save the data recipe and model under the variable: `nps_rec` and `nps_mod`, respectively.

**Code you your models in the cell below.**

In [19]:
# parallel slopes model

ps_rec = recipe(income ~ hs + urbanization, data = hate_crimes) |> 
    step_naomit(everything()) |>   # remove missing values
    step_dummy(urbanization) |>    # dummy encode the urbanization variable
    prep()                         # run the recipe on the training data provided

mod = linear_reg() |> set_engine('lm')

ps_mod = mod |> fit(income ~ ., juice(ps_rec))

ps_mod

ERROR: Error: object 'hate_crimes' not found


In [12]:
library(tidymodels)
ps_rec <- recipe(income ~ hs + urbanization, data = hate_crimes) %>%
  step_normalize(all_predictors())

ps_mod <- linear_reg() %>%
  set_engine("lm") %>%
  fit(income ~ hs + urbanization, data = hate_crimes)

nps_rec <- recipe(income ~ hs * urbanization, data = hate_crimes) %>%
  step_normalize(all_predictors())

nps_mod <- linear_reg() %>%
  set_engine("lm") %>%
  fit(income ~ hs * urbanization, data = hate_crimes)

tidy(ps_mod)
tidy(nps_mod)

── Attaching packages ────────────────────────────────────── tidymodels 1.3.0 ──

✔ broom        1.0.8     ✔ rsample      1.3.0
✔ dials        1.4.0     ✔ tibble       3.2.1
✔ dplyr        1.1.4     ✔ tidyr        1.3.1
✔ infer        1.0.8     ✔ tune         1.3.0
✔ modeldata    1.4.0     ✔ workflows    1.2.0
✔ parsnip      1.3.1     ✔ workflowsets 1.1.0
✔ purrr        1.0.4     ✔ yardstick    1.3.2
✔ recipes      1.3.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()



ERROR: Error: object 'hate_crimes' not found


The following code creates a table of your model predictions over the training data. Calculate the [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) (R<sup>2</sup>) for each model:

```r
rbind(
    augment(ps_mod, juice(ps_rec)) |> select(income, .pred, .resid) |> mutate(model = "parallel_slopes"),
    augment(nps_mod, juice(nps_rec)) |> select(income, .pred, .resid) |> mutate(model = "interaction")
)
```

1. Group rows by the `model` variable (use the `group_by` function).
1. Calculate the variance of income over the variance of your predictions for each model using the "grouped" data from the step above (use `summarize(r_squared = var(.pred) / var(income))`).

In [13]:
library(dplyr)

model_results <- bind_rows(
  hate_crimes %>% 
    mutate(model = "Parallel Slopes", .pred = predict(ps_mod, new_data = hate_crimes)$.pred),
  hate_crimes %>% 
    mutate(model = "Non-Parallel Slopes", .pred = predict(nps_mod, new_data = hate_crimes)$.pred)
)

model_results %>%
  group_by(model) %>%
  summarize(r_squared = var(.pred) / var(income))



ERROR: Error: object 'hate_crimes' not found


🎶 Note: you can also use the `glance` function with a model as input to find the coefficient of determination.

In [14]:
glance(ps_mod)

ERROR: Error: object 'ps_mod' not found


❓Compare the adjusted proportion of variance accounted for in each model. Based on this comparison, which model do you prefer? Why? 

**Answer:**

In [15]:
# I preder the non parallel slopes model because it has better explanations for variations in income.



❓Using your preferred model, based on your regression model parameters (and the data visualizations), is `income` greater in states that have lower or higher levels of `urbanization`? By how much?

**Hint:** use the `tidy` function with your model as input to access the parameters in a nice table.

**Answer:**

In [16]:

#Income is lower in states with higher urbanization.


❓For every one percentage point increase of high-school educated adults in a state (`hs` variable), what is the associated average increase in `income`?

**Answer:**

In [17]:

#It would be 1%, I think?